`conda env create -n caiman_env -f optinist/conda/caiman_env.yaml`

In [ ]:
"""
Setup:
1. Create environment:
    In terminal, run:
   conda env create -n caiman_test_env -f studio/app/optinist/wrappers/caiman/conda/caiman.yaml
2. Activate environment:
   conda activate caiman_env
   pip install pynwb imageio ipython jupyter notebook "pydantic<2.0.0" python-dotenv uvicorn xmltodict fastapi "python-jose[cryptography]" passlib python-multipart bcrypt 
  - If running in VS code, you may need to restart and/or select the correct environment with "Python: Select Interpreter"
3. Run this notebook
"""

In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))

In [2]:
from studio.app.optinist.wrappers.caiman import motion_correction, cnmf
from studio.app.common.dataclass import ImageData

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
sample_data = ImageData(["../sample_data/input/sample_mouse2p_image.tiff"])

In [4]:
motion_correction_params = {
    'border_nan': 'copy', 
    'gSig_filt': None, 
    'is3D': False, 
    'max_deviation_rigid': 3, 
    'max_shifts': [6, 6], 
    'min_mov': None, 
    'niter_rig': 1, 
    'nonneg_movie': True, 
    'num_frames_split': 80, 
    'num_splits_to_process_els': None, 
    'num_splits_to_process_rig': None, 
    'overlaps': [32, 32], 
    'pw_rigid': False, 
    'shifts_opencv': True, 
    'splits_els': 14, 
    'splits_rig': 14, 
    'strides': [96, 96], 
    'upsample_factor_grid': 4, 
    'use_cuda': False
}

In [5]:
ret = motion_correction.caiman_mc(sample_data, motion_correction_params)

2025-01-24 18:57:43.987817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: expected string or bytes-like object

In [ ]:
caiman_cnmf_params = {
    'p': 1,
    'nb': 2,
    'merge_thr': 0.85,
    'rf': None,
    'stride': 6,
    'K': 4,
    'gSig': [4, 4], 
    'method_init': 'greedy_roi',
    'ssub': 1,
    'tsub': 1,
    'thr': 0.9,
}

In [ ]:
ret = cnmf.caiman_cnmf(ret['mc_images'], caiman_cnmf_params)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot figure
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=('Mean Image with ROIs', 'Components Spatial', 
                                  'Components Temporal', 'Residuals'))

# Plot mean image with ROIs overlay
Cn = local_correlations(mc_images.transpose(1, 2, 0))
fig.add_trace(
    go.Heatmap(z=Cn, colorscale='gray'),
    row=1, col=1
)

# Add ROI contours
for i in range(cnm.estimates.A.shape[1]):
    roi = cnm.estimates.A[:, i].toarray().reshape(dims, order='F')
    if i in cnm.estimates.idx_components:
        color = 'red'  # Good components
    else:
        color = 'blue' # Rejected components
    fig.add_trace(
        go.Contour(z=roi, showscale=False, 
                   contours=dict(coloring='lines', 
                   showlabels=True),
                   line=dict(color=color)),
        row=1, col=1
    )

# Plot spatial components
fig.add_trace(
    go.Heatmap(z=cnm.estimates.A.toarray().reshape((*dims, -1))[:,:,0],
               colorscale='viridis'),
    row=1, col=2
)

# Plot temporal components 
fig.add_trace(
    go.Scatter(y=cnm.estimates.C[0], mode='lines'),
    row=2, col=1
)

# Plot residuals
fig.add_trace(
    go.Heatmap(z=cnm.estimates.b.reshape(dims, order='F'),
               colorscale='viridis'),
    row=2, col=2
)

fig.update_layout(height=800, width=1000, title_text="CNMF Results")
fig.show()